In [25]:
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.model_selection import cross_val_score

In [26]:
df = pd.read_csv('dataset_ZavarzinE_(baseline).csv', index_col=False)
y_cols = ['химшлак последний Al2O3', 'химшлак последний CaO', 
          'химшлак последний FeO', 'химшлак последний MgO',
          'химшлак последний MnO', 'химшлак последний R',
          'химшлак последний SiO2']
X = df.drop(columns=y_cols)
y = df.filter(y_cols)
del df
print('X:', X.shape)
print('y', y.shape)

X: (6238, 53)
y (6238, 7)


In [27]:
mape_scorer = make_scorer(lambda y, y_pred: (1-mean_absolute_percentage_error(y, y_pred))*100)
report = []
def check_model(estimator, name):
    results = {y_col: cross_val_score(estimator, X, y[y_col], cv=5, n_jobs=-1,
                           scoring=mape_scorer).mean() for y_col in tqdm(y_cols) }
    report.append([name] + list(results.values()))
    return results

In [28]:
check_model(LinearRegression(), 'LinearRegression')

100%|██████████| 7/7 [00:01<00:00,  3.71it/s]


{'химшлак последний Al2O3': 90.80618973362897,
 'химшлак последний CaO': 95.12679883894958,
 'химшлак последний FeO': 64.93017569467885,
 'химшлак последний MgO': 73.98370934773823,
 'химшлак последний MnO': 69.78851329833415,
 'химшлак последний R': 93.7125674615406,
 'химшлак последний SiO2': 94.32380165854781}

In [29]:
check_model(RandomForestRegressor(), 'RandomForestRegressor')

100%|██████████| 7/7 [04:39<00:00, 39.88s/it]


{'химшлак последний Al2O3': 91.14770612801534,
 'химшлак последний CaO': 95.14795186628295,
 'химшлак последний FeO': 64.21901355749671,
 'химшлак последний MgO': 73.29469166948668,
 'химшлак последний MnO': 66.42433968739755,
 'химшлак последний R': 93.8572538974257,
 'химшлак последний SiO2': 94.55741292911414}

In [30]:
check_model(GradientBoostingRegressor(), 'GradientBoostingRegressor')

100%|██████████| 7/7 [01:03<00:00,  9.03s/it]


{'химшлак последний Al2O3': 90.96351208451813,
 'химшлак последний CaO': 95.14088890183746,
 'химшлак последний FeO': 64.74660981674963,
 'химшлак последний MgO': 73.4007892999197,
 'химшлак последний MnO': 68.57297955943699,
 'химшлак последний R': 93.94703334015468,
 'химшлак последний SiO2': 94.41620250914985}

In [32]:
with open('report.csv', 'w') as report_file:
    report_file.writelines(['\t'.join(map(str, report_line)).replace('.', ',')+'\n' for report_line in report])